# This notebook implements automatic generation of ReconConfig xml files to be used to run MuhRec in a loop
### pathname, filename and parameters are given as examples and need to be adjusted for custom application

In [1]:
import xml.etree.ElementTree as ET
import numpy
import os

In [ ]:

# Open original/template ReconConfig file
et = ET.parse('D:/Kaestner/ISIS_IAEAData/RB1730044/PSI_Cylinder/Sample/RB1730044/Tomo_test/CT_recon_WhiteBeam_BBcorrected_const_dose/ReconConfig.xml')
root = et.getroot()


#with this I find all modules in the pre-processing chain
for module in root.findall('./processchain/preprocessing/module'):
    name= module.find('modulename')
    print(name.text)


# Find the first module (BBmodule)
BBmodule = root.find('./processchain/preprocessing/module/modulename') ## find the first one, which is always the BBLogNorm
# Find relevant parameters that I want to update, ex. names and name mask for BB external images
BB_OB_ext_name=root.find('./processchain/preprocessing/module/parameters/BB_OB_ext_name')
BB_sample_ext_name=root.find('./processchain/preprocessing/module/parameters/BB_sample_ext_name')

# Find other items to be changed for each file
filemask = root.find('./projections/filemask')
outputpath = root.find('./matrix/path')
obmask = root.find('./projections/obfilemask')
obfirstindex = root.find('./projections/obfirstindex')
projpath = root.find('./projections/path')
projrefpath = root.find('./projections/referencepath')




# Loop over the bins to create the ReconConfig
first_bin=0
last_bin=165
for i in range(first_bin, last_bin):
    #destination path to save the xml files 
    destpath=r"D:Kaestner/ISIS_IAEAData/RB1730044/PSI_Cylinder/Sample/RB1730044/Tomo_test/CT_recon_binSum_165bin_249_BBcorrected_const_dose" 
    path = r"/bin"+str(i).zfill(4)+"/"
    #update the BB parameters (filenames)
    BB_OB_ext_name.text = 'D:/Kaestner/ISIS_IAEAData/RB1730044/PSI_Cylinder/Sample/RB1730044/Tomo_test/Sample/Backgrounds/ConstantBackground_binned/Bin_'+str(i).zfill(4)+'/BackGround_0000.fits'
    BB_sample_ext_name.text = 'D:/Kaestner/ISIS_IAEAData/RB1730044/PSI_Cylinder/Sample/RB1730044/Tomo_test/Sample/Backgrounds/ConstantBackground_binned/Bin_'+str(i).zfill(4)+'/BackGround_####.fits'
    #update the filemask and other parameters
    filemask.text = 'D:/Kaestner/ISIS_IAEAData/RB1730044/PSI_Cylinder/Sample/RB1730044/Tomo_test/Sample/angle####_Corrected_Binned\IMAT_tomo_binned_'+str(i).zfill(5)+'.fits'
    outputpath.text = destpath+path
    obmask.text = 'D:/Kaestner/ISIS_IAEAData/RB1730015/Tomo_test_RB1730044/OpenBeam_aft#_Corrected_Binned/IMAT_tomo_binned_'+str(i).zfill(5)+'.fits'
    obfirstindex.text=str(2)
    projpath.text=' '
    projrefpath.text=' '

    #path to save the config file
    fileconfigpath = 'D:/Kaestner/ISIS_IAEAData/RB1730044/PSI_Cylinder/Sample/RB1730044/Tomo_test/CT_configurations_BBconst_dose'
    #file name
    fileconfigname = '/ReconConfig_bin'+str(i).zfill(4)+'.xml'
    #write the file
    et.write(fileconfigpath+fileconfigname) # It works!!
    #create the destination path for the reconstructed tomo (this is however re-checked in RunMuhRec)
    if not os.path.exists(destpath+path):
        os.makedirs(destpath+path)